In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import string
import pandas as pd
import re
import numpy as np
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import pos_tag
from sklearn.tree import DecisionTreeClassifier
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Ahmed
[nltk_data]     M.Sallam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df  = pd.read_csv("Spam_Email_Data.csv")
def data_cleaning(text):
    clean_text = re.sub('<.*?>', '', text) # remove HTML tags
    clean_text = re.sub(r'[^\w\s]', '', clean_text)  # remove punctuation
    clean_text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '', clean_text)# remove dates
    clean_text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', clean_text)# remove email addresses
    return clean_text

df['text'] = df['text'].apply(data_cleaning)

df

,text,target
0,From ilugadminlinuxie Mon Jul 29 112802 2002 R...,0
1,From gort44excitecom Mon Jun 24 175421 2002 Re...,1
2,From forkadminxentcom Mon Jul 29 113957 2002 R...,1
3,From dcm123btamailnetcn Mon Jun 24 174923 2002...,1
4,From ilugadminlinuxie Mon Aug 19 110247 2002 R...,0
...,...,...
5791,From ilugadminlinuxie Mon Jul 22 181245 2002 R...,0
5792,From forkadminxentcom Mon Oct 7 203702 2002 Re...,0
5793,Received from hqpronsnet localhost 127001 by h...,1
5794,From razorusersadminlistssourceforgenet Thu Se...,0


In [3]:
def text_processing(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Stop word removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

   # Part-of-speech tagging
    pos_tokens = pos_tag(filtered_tokens)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in pos_tokens]




    processed_text = ' '.join([token for token, pos in lemmatized_tokens])
    return processed_text

In [4]:
final = []


def model_accuracy(model_name, y_test, y_pred):
    accuracy_test = accuracy_score(y_test, y_pred)
    precision_test = precision_score(y_test, y_pred)
    recall_test = recall_score(y_test, y_pred)
    f1_test = f1_score(y_test, y_pred)

    final.append({
        'model': model_name,
        'accuracy_test': accuracy_test,
        'precision_test': precision_test,
        # 'recall_test': recall_test,
        # 'f1_test': f1_test
    })

    return accuracy_test, precision_test #, recall_test, f1_test


In [5]:
x_train , x_test , y_train , y_test = train_test_split(df['text'] , df['target'] , test_size=0.4 , random_state=50)

sentences = [row.split() for row in x_train]
word2vec_model = Word2Vec(sentences , vector_size = 100)


def get_word2vec_embedding(text):
    words = text.split()
    embeding =  np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv], axis=0)
    return embeding


x_train_w2v = np.array([get_word2vec_embedding(text) for text in x_train])
x_test_w2v  = np.array([get_word2vec_embedding(text) for text in x_test])



In [6]:
# Create a logistic regression model
logistic_model = LogisticRegression(C=0.5, random_state=50)


logistic_model.fit(x_train_w2v , y_train)


y_pred_log = logistic_model.predict(x_test_w2v)

accuracy_w2v, precision_w2v = model_accuracy("Logistic Regression (w2v)", y_test, y_pred_log)



In [7]:
# TF-IDF Vectorizer
tfidf = TfidfVectorizer()
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

# Bag of Words (BoW) Vectorizer
bow = CountVectorizer()
x_train_bow = bow.fit_transform(x_train)
x_test_bow = bow.transform(x_test)



In [8]:
# Train logistic regression model using TF-IDF features
logistic_model_tfidf = LogisticRegression(C=0.5, random_state=50)
logistic_model_tfidf.fit(x_train_tfidf, y_train)
y_pred_log_tfidf = logistic_model_tfidf.predict(x_test_tfidf)


In [9]:
y_pred_log_tfidf

array([1, 0, 1, ..., 1, 0, 1], dtype=int64)

In [10]:
# Train logistic regression model using BoW features
logistic_model_bow = LogisticRegression(C=0.5, random_state=50)
logistic_model_bow.fit(x_train_bow, y_train)
y_pred_log_bow = logistic_model_bow.predict(x_test_bow)


c:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
y_pred_log_bow

array([1, 0, 1, ..., 1, 0, 1], dtype=int64)

In [12]:
# Evaluate models
model_accuracy("Logistic Regression (TF-IDF)", y_test, y_pred_log_tfidf)
model_accuracy("Logistic Regression (BoW)", y_test, y_pred_log_bow)



(0.9948253557567918, 0.994579945799458)

In [13]:
# Display results
results_df = pd.DataFrame(final)
print(results_df)

                          model  accuracy_test  precision_test
0     Logistic Regression (w2v)       0.982751        0.984807
1  Logistic Regression (TF-IDF)       0.969815        0.997041
2     Logistic Regression (BoW)       0.994825        0.994580


In [14]:
# Doc2Vec


# Tag documents
td_train = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[str(i)]) for i, doc in enumerate(x_train)]
td_test = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[str(i)]) for i, doc in enumerate(x_test)]


In [15]:
# Train Doc2Vec model
doc2vec_model = Doc2Vec(td_train, vector_size=100, window=5, min_count=1, workers=4)

x_train_doc2vec = np.array([doc2vec_model.infer_vector(doc.words) for doc in td_train])
x_test_doc2vec = np.array([doc2vec_model.infer_vector(doc.words) for doc in td_test])


In [16]:
# Train logistic regression model using Doc2Vec features with increased max_iter
dec2vec_logistic_model = LogisticRegression(C=0.5, random_state=50, max_iter=1000)  # Increase max_iter
dec2vec_logistic_model.fit(x_train_doc2vec, y_train)
y_pred_log_doc2vec = dec2vec_logistic_model.predict(x_test_doc2vec)

In [17]:
# Evaluate model
accuracy_doc2vec, precision_doc2vec = model_accuracy("Logistic Regression (Doc2Vec)", y_test, y_pred_log_doc2vec)

# # Display results
# final.append({
#     'model': "Logistic Regression (Doc2Vec)",
#     'accuracy_test': accuracy_doc2vec,
#     'precision_test': precision_doc2vec,
#     'recall_test': recall_doc2vec,
#     'f1_test': f1_doc2vec
# })
# results_df = pd.DataFrame(final)
# print(results_df)


In [18]:
# Decision Tree with Word2Vec
decision_tree_w2v = DecisionTreeClassifier(random_state=50)
decision_tree_w2v.fit(x_train_w2v, y_train)
y_pred_dt_w2v = decision_tree_w2v.predict(x_test_w2v)
model_accuracy("Decision Tree (Word2Vec)", y_test, y_pred_dt_w2v)


(0.9534282018111255, 0.9260752688172043)

In [19]:
# Decision Tree with Doc2Vec
decision_tree_doc2vec = DecisionTreeClassifier(random_state=50)
decision_tree_doc2vec.fit(x_train_doc2vec, y_train)
y_pred_dt_doc2vec = decision_tree_doc2vec.predict(x_test_doc2vec)
model_accuracy("Decision Tree (Doc2Vec)", y_test, y_pred_dt_doc2vec)

(0.8801207416990082, 0.8093333333333333)

In [20]:
# Decision Tree with TF-IDF
decision_tree_tfidf = DecisionTreeClassifier(random_state=50)
decision_tree_tfidf.fit(x_train_tfidf, y_train)
y_pred_dt_tfidf = decision_tree_tfidf.predict(x_test_tfidf)
model_accuracy("Decision Tree (TF-IDF)", y_test, y_pred_dt_tfidf)


(0.9780077619663649, 0.9564068692206077)

In [21]:
# Decision Tree with Bag of Words
decision_tree_bow = DecisionTreeClassifier(random_state=50)
decision_tree_bow.fit(x_train_bow, y_train)
y_pred_dt_bow = decision_tree_bow.predict(x_test_bow)
model_accuracy("Decision Tree (BoW)", y_test, y_pred_dt_bow)



(0.9793014230271668, 0.9602122015915119)

In [22]:
# Display results
results_df = pd.DataFrame(final)
print(results_df)


                           model  accuracy_test  precision_test
0      Logistic Regression (w2v)       0.982751        0.984807
1   Logistic Regression (TF-IDF)       0.969815        0.997041
2      Logistic Regression (BoW)       0.994825        0.994580
3  Logistic Regression (Doc2Vec)       0.967658        0.973050
4       Decision Tree (Word2Vec)       0.953428        0.926075
5        Decision Tree (Doc2Vec)       0.880121        0.809333
6         Decision Tree (TF-IDF)       0.978008        0.956407
7            Decision Tree (BoW)       0.979301        0.960212
